## Create CNN model and optimize it using keras Tuner


In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 6.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=6973c109079a43317fffdf06ded256e0266b5e9c1b8c68e200dd2747d0303a91
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=fa86658652d09bdc8c2b7c97032661bd669181f81ce3296ff3ce9cb8404b0f23
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

### to tune our model tensorflow version should be greater than 2.0


In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0


In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
train_images[0].shape

(28, 28, 1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5089 - accuracy: 0.8283 - val_loss: 0.3592 - val_accuracy: 0.8677
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3639 - accuracy: 0.8648 - val_loss: 0.3910 - val_accuracy: 0.8570
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3271 - accuracy: 0.8808 - val_loss: 0.3624 - val_accuracy: 0.8657


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4103 - accuracy: 0.8507 - val_loss: 0.3378 - val_accuracy: 0.8722
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2661 - accuracy: 0.9022 - val_loss: 0.2674 - val_accuracy: 0.9003
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2107 - accuracy: 0.9215 - val_loss: 0.2675 - val_accuracy: 0.9038


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4098 - accuracy: 0.8546 - val_loss: 0.3232 - val_accuracy: 0.8830
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2670 - accuracy: 0.9023 - val_loss: 0.2749 - val_accuracy: 0.9015
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2103 - accuracy: 0.9228 - val_loss: 0.2551 - val_accuracy: 0.9070


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5473 - accuracy: 0.8132 - val_loss: 0.4133 - val_accuracy: 0.8497
Epoch 2/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3916 - accuracy: 0.8566 - val_loss: 0.3916 - val_accuracy: 0.8552
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3771 - accuracy: 0.8616 - val_loss: 0.4290 - val_accuracy: 0.8387


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3954 - accuracy: 0.8584 - val_loss: 0.3094 - val_accuracy: 0.8792
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2637 - accuracy: 0.9028 - val_loss: 0.2723 - val_accuracy: 0.8970
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2100 - accuracy: 0.9221 - val_loss: 0.2745 - val_accuracy: 0.9022


INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                589856    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 599,754
Trainable params: 599,754
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1649 - accuracy: 0.9389 - val_loss: 0.2847 - val_accuracy: 0.9008
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1279 - accuracy: 0.9523 - val_loss: 0.2822 - val_accuracy: 0.9093
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0996 - accuracy: 0.9630 - val_loss: 0.2920 - val_accuracy: 0.9080
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0776 - accuracy: 0.9718 - val_loss: 0.3293 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0573 - accuracy: 0.9786 - val_loss: 0.3849 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0491 - accuracy: 0.9819 - val_loss: 0.4152 - val_accuracy: 0.9048
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0378 - accuracy: 0.9871 - val_loss: 0.4661 - val_accuracy